In [1]:
import pygame
import sys
import math  # For square root and trigonometric functions

# Initialize pygame
pygame.init()

# Window settings
WIDTH, HEIGHT = 800, 600
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (62, 88, 148)
GREEN = (152, 186, 87)
YELLOW = (224, 191, 101)
GRAY = (200, 200, 200)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Paint')
screen.fill(WHITE)
clock = pygame.time.Clock()

# Drawing settings
drawing = False
last_pos = None
color = BLACK
brush_size = 5
mode = "brush"
start_pos = None

# Button layout
buttons = {
    "brush": pygame.Rect(10, 10, 80, 30),
    "rect": pygame.Rect(100, 10, 80, 30),
    "circle": pygame.Rect(190, 10, 80, 30),
    "eraser": pygame.Rect(280, 10, 80, 30),
    "square": pygame.Rect(10, 50, 80, 30),
    "right_tri": pygame.Rect(100, 50, 80, 30),
    "equil_tri": pygame.Rect(190, 50, 80, 30),
    "rhombus": pygame.Rect(280, 50, 80, 30),
    "black": pygame.Rect(370, 10, 30, 30),
    "yellow": pygame.Rect(410, 10, 30, 30),
    "green": pygame.Rect(450, 10, 30, 30),
    "blue": pygame.Rect(490, 10, 30, 30),
    "increase": pygame.Rect(530, 10, 30, 30),
    "decrease": pygame.Rect(570, 10, 30, 30)
}

# Draw toolbar buttons
def draw_buttons():
    pygame.draw.rect(screen, GRAY, buttons["brush"])
    pygame.draw.rect(screen, GRAY, buttons["rect"])
    pygame.draw.rect(screen, GRAY, buttons["circle"])
    pygame.draw.rect(screen, GRAY, buttons["eraser"])
    pygame.draw.rect(screen, GRAY, buttons["square"])
    pygame.draw.rect(screen, GRAY, buttons["right_tri"])
    pygame.draw.rect(screen, GRAY, buttons["equil_tri"])
    pygame.draw.rect(screen, GRAY, buttons["rhombus"])
    pygame.draw.rect(screen, BLACK, buttons["black"])
    pygame.draw.rect(screen, YELLOW, buttons["yellow"])
    pygame.draw.rect(screen, GREEN, buttons["green"])
    pygame.draw.rect(screen, BLUE, buttons["blue"])
    pygame.draw.rect(screen, GRAY, buttons["increase"])
    pygame.draw.rect(screen, GRAY, buttons["decrease"])

    font = pygame.font.Font(None, 20)
    screen.blit(font.render("Brush", True, BLACK), (20, 15))
    screen.blit(font.render("Rect", True, BLACK), (115, 15))
    screen.blit(font.render("Circle", True, BLACK), (200, 15))
    screen.blit(font.render("Eraser", True, BLACK), (290, 15))
    screen.blit(font.render("Square", True, BLACK), (15, 55))
    screen.blit(font.render("Right", True, BLACK), (110, 55))
    screen.blit(font.render("Equil", True, BLACK), (200, 55))
    screen.blit(font.render("Rhomb", True, BLACK), (290, 55))
    screen.blit(font.render("+", True, BLACK), (540, 15))
    screen.blit(font.render("-", True, BLACK), (580, 15))

draw_buttons()

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if event.type == pygame.MOUSEBUTTONDOWN:
            x, y = event.pos
            # Check button clicks
            for btn, rect in buttons.items():
                if rect.collidepoint(x, y):
                    if btn in ["brush", "rect", "circle", "eraser", "square", "right_tri", "equil_tri", "rhombus"]:
                        mode = btn
                    elif btn == "black":
                        color = BLACK
                    elif btn == "yellow":
                        color = YELLOW
                    elif btn == "green":
                        color = GREEN
                    elif btn == "blue":
                        color = BLUE
                    elif btn == "increase":
                        brush_size += 2
                    elif btn == "decrease" and brush_size > 2:
                        brush_size -= 2
                    break
            else:
                drawing = True
                last_pos = event.pos
                start_pos = event.pos

        if event.type == pygame.MOUSEBUTTONUP:
            drawing = False
            if start_pos:
                end_pos = event.pos
                x1, y1 = start_pos
                x2, y2 = end_pos
                width = x2 - x1
                height = y2 - y1

                # Rectangle
                if mode == "rect":
                    pygame.draw.rect(screen, color, (x1, y1, width, height), brush_size)

                # Circle
                elif mode == "circle":
                    radius = int(((width) ** 2 + (height) ** 2) ** 0.5)
                    pygame.draw.circle(screen, color, start_pos, radius, brush_size)

                # Square (based on smallest side)
                elif mode == "square":
                    side = min(abs(width), abs(height))
                    pygame.draw.rect(screen, color, (x1, y1, side * (1 if width > 0 else -1), side * (1 if height > 0 else -1)), brush_size)

                # Right triangle
                elif mode == "right_tri":
                    pygame.draw.polygon(screen, color, [(x1, y1), (x1, y2), (x2, y2)], brush_size)

                # Equilateral triangle
                elif mode == "equil_tri":
                    side = math.hypot(width, height)
                    height_tri = side * math.sqrt(3) / 2
                    points = [
                        (x1, y1),
                        (x1 + side / 2, y1 + height_tri),
                        (x1 - side / 2, y1 + height_tri)
                    ]
                    pygame.draw.polygon(screen, color, points, brush_size)

                # Rhombus (centered between start and end)
                elif mode == "rhombus":
                    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
                    dx = abs(x2 - x1) // 2
                    dy = abs(y2 - y1) // 2
                    points = [
                        (cx, cy - dy),
                        (cx + dx, cy),
                        (cx, cy + dy),
                        (cx - dx, cy)
                    ]
                    pygame.draw.polygon(screen, color, points, brush_size)

            start_pos = None

        # Free drawing (brush and eraser)
        if event.type == pygame.MOUSEMOTION and drawing:
            if mode == "brush":
                pygame.draw.line(screen, color, last_pos, event.pos, brush_size)
                last_pos = event.pos
            elif mode == "eraser":
                if not any(rect.collidepoint(event.pos) for rect in buttons.values()):
                    pygame.draw.line(screen, WHITE, last_pos, event.pos, brush_size)
                    last_pos = event.pos

    draw_buttons()
    pygame.display.flip()
    clock.tick(60)


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\S\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
